# Classification Project: EDA → Preprocessing → Train/Test → Model → Threshold → Save/Load

Мы используем:
- **pandas / numpy**: загрузка, подготовка признаков
- **matplotlib / seaborn**: минимальный EDA
- **scikit-learn**: `train_test_split`, `Pipeline`, `ColumnTransformer`, `SimpleImputer`, `OneHotEncoder`, `StandardScaler`
- **модели**: `LogisticRegression`, `DecisionTreeClassifier`, `RandomForestClassifier`, `GradientBoostingClassifier`
- **метрики**: `confusion_matrix`, `precision/recall/F1`, `ROC-AUC`
- **CV + GridSearch**: `StratifiedKFold`, `cross_val_score`, `GridSearchCV`
- **порог**: 0.5 vs другой
- **сохранение**: `joblib.dump` / `joblib.load`

## Итог
В конце ноутбука вы получите:
1) обученную модель в одном объекте `Pipeline`  
2) честную оценку на test  
3) сохранённую модель `*.joblib`  
4) проверку, что загруженная модель предсказывает так же


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, log_loss
)

import joblib

sns.set_context("notebook")
np.random.seed(42)
print("Ready!")

# 1) Загрузка датасета

## Ваш Kaggle CSV
1) скачайте `train.csv` (или другой csv)  
2) положите рядом с ноутбуком или укажите путь  
3) выставьте `TARGET_COL`


In [ ]:
# === Настройте под ваш датасет ===
DATA_PATH = "train.csv"   # например: "kaggle_train.csv"
TARGET_COL = None         # например: "Survived" или "Outcome"
ID_COL = None             # например: "Id" (если есть и не нужен)

df = None

try:
    df = pd.read_csv(DATA_PATH)
    print("Loaded CSV:", DATA_PATH, "| shape:", df.shape)
except Exception as e:
    print("Could not read CSV:", e)
    print("Falling back to demo dataset...")

if df is None:
    from sklearn.datasets import load_breast_cancer
    data = load_breast_cancer(as_frame=True)
    df = data.frame.copy()
    df.rename(columns={"target": "target"}, inplace=True)
    TARGET_COL = "target"

    # Категориальный признак из числового (демо OneHotEncoder)
    df["radius_bin"] = pd.qcut(df["mean radius"], q=4, labels=["low", "mid_low", "mid_high", "high"])

    # Добавим пропуски (демо SimpleImputer)
    df.loc[df.sample(frac=0.06, random_state=42).index, "mean texture"] = np.nan
    df.loc[df.sample(frac=0.04, random_state=7).index, "radius_bin"] = np.nan

    print("Loaded demo Breast Cancer | shape:", df.shape)

df.head()

# 2) Определяем target и приводим его к формату 0/1

Для бинарной классификации target должен быть:
- либо 0/1
- либо строки/категории, которые можно замапить в 0/1

Пример:
```python
mapping = {"No": 0, "Yes": 1}
df[TARGET_COL] = df[TARGET_COL].map(mapping)
```

Если target не бинарный (3+ классов), этот шаблон можно адаптировать, но сейчас держим бинарный кейс.


In [ ]:
if TARGET_COL is None:
    print("⚠️ TARGET_COL = None. Укажите целевую колонку (например, 'Survived').")
else:
    print("TARGET_COL:", TARGET_COL)
    print("Unique target values (first 20):", df[TARGET_COL].dropna().unique()[:20])
    display(df[TARGET_COL].value_counts(dropna=False))

# 3) Быстрый обзор таблицы

Зачем:
- размер и типы
- сколько пропусков
- пример строк


In [ ]:
print("Shape:", df.shape)
display(df.head(3))

print("\nDtypes summary:")
display(df.dtypes.value_counts())

print("\nMissing share (top 15):")
na_share = df.isna().mean().sort_values(ascending=False)
display(na_share.head(15))

# 4) Мини-EDA (matplotlib + seaborn)

Обязательный минимум:
1) баланс классов  
2) распределение 1 числового признака по классам  
3) countplot 1 категориального признака (если есть)  
4) корреляции числовых с target (если target 0/1)


In [ ]:
assert TARGET_COL in df.columns, "Проверьте TARGET_COL — он должен быть в df.columns"

# 4.1) Баланс классов
plt.figure(figsize=(5,3))
df[TARGET_COL].value_counts().plot(kind="bar")
plt.title("Target class counts")
plt.ylabel("count")
plt.grid(True, axis="y")
plt.show()

# 4.2) Выберем один числовой признак и нарисуем распределение по классам
num_cols = df.select_dtypes(include="number").columns.tolist()
num_cols = [c for c in num_cols if c != TARGET_COL]

if len(num_cols) > 0:
    col = num_cols[0]
    plt.figure(figsize=(7,4))
    for t in sorted(df[TARGET_COL].dropna().unique())[:2]:
        subset = df[df[TARGET_COL] == t][col].dropna()
        plt.hist(subset, bins=25, alpha=0.5, label=f"target={t}")
    plt.title(f"Feature distribution by class: {col}")
    plt.xlabel(col)
    plt.ylabel("count")
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print("No numeric columns found.")

# 4.3) Если есть категориальные — countplot
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
cat_cols = [c for c in cat_cols if c != TARGET_COL]

if len(cat_cols) > 0:
    c = cat_cols[0]
    plt.figure(figsize=(8,3))
    sns.countplot(data=df, x=c, hue=TARGET_COL)
    plt.title(f"Count plot: {c} by target")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, axis="y")
    plt.show()
else:
    print("No categorical columns found (or only target).")

# 4.4) Корреляции числовых признаков с target
if len(num_cols) > 0 and df[TARGET_COL].dropna().nunique() <= 2:
    corr = df[num_cols + [TARGET_COL]].corr(numeric_only=True)[TARGET_COL].drop(TARGET_COL)
    top = corr.abs().sort_values(ascending=False).head(10)
    display(pd.DataFrame({"corr": corr.loc[top.index], "abs_corr": top.values}).sort_values("abs_corr", ascending=False))

    cols_hm = top.index.tolist() + [TARGET_COL]
    plt.figure(figsize=(9,6))
    sns.heatmap(df[cols_hm].corr(numeric_only=True), annot=False)
    plt.title("Correlation heatmap (top numeric + target)")
    plt.show()

# 5) Формируем X/y и делаем train/test split

Правила:
- `y` = target
- `X` = все остальные признаки
- id-колонку убрать (если есть)
- `stratify=y` для похожего баланса классов


In [ ]:
df_work = df.copy()

if ID_COL is not None and ID_COL in df_work.columns:
    df_work = df_work.drop(columns=[ID_COL])
    print("Dropped ID_COL:", ID_COL)

# Убираем строки без target
df_work = df_work.dropna(subset=[TARGET_COL]).copy()

X = df_work.drop(columns=[TARGET_COL])
y = df_work[TARGET_COL]

# Если target не 0/1, замапьте его ДО split!
# y = y.map({"No":0, "Yes":1})

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train share:", y_train.value_counts(normalize=True).to_dict())
print("Test  share:", y_test.value_counts(normalize=True).to_dict())

# 6) Preprocessing через Pipeline + ColumnTransformer

Универсальный шаблон для Kaggle:
- **числовые**: медиана + StandardScaler (полезно для логрег)
- **категориальные**: most_frequent + OneHotEncoder

Деревьям scaler не нужен, но он не мешает, потому что лес/бустинг увидят уже числовой one-hot.


In [ ]:
cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
num_features = [c for c in X_train.columns if c not in cat_features]

print("Numeric features:", len(num_features))
print("Categorical features:", len(cat_features))
print("Example cat:", cat_features[:5])

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer([
    ("num", num_pipe, num_features),
    ("cat", cat_pipe, cat_features),
], remainder="drop")

preprocess

## Метрики и порог

- `predict_proba` → вероятности класса 1  
- порог 0.5 vs другой: управляем precision/recall
- `log_loss` оценивает качество вероятностей (уверенности)


In [ ]:
def metrics_at_threshold(y_true, p1, thr=0.5):
    y_pred = (p1 >= thr).astype(int)
    return {
        "threshold": thr,
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, p1),
        "logloss": log_loss(y_true, p1),
    }, y_pred

# 7) Baseline: LogisticRegression

Быстро обучается, даёт вероятности — отличный baseline.


In [ ]:
lr_model = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=3000))
])

lr_model.fit(X_train, y_train)

proba_test = lr_model.predict_proba(X_test)[:, 1]
m05, y_pred_05 = metrics_at_threshold(y_test.values, proba_test, thr=0.5)

print("LogReg metrics @0.5:", m05)
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred_05))
print("\nReport:\n", classification_report(y_test, y_pred_05, digits=4))

## 7.1) Пороговая таблица (0.5 vs другие)

Смотрим, как меняются precision/recall/F1 при разных порогах.


In [ ]:
thresholds = [0.2, 0.35, 0.5, 0.65, 0.8]
rows = []
for t in thresholds:
    m, _ = metrics_at_threshold(y_test.values, proba_test, thr=t)
    rows.append(m)

pd.DataFrame(rows)

## 7.2) ROC curve

ROC-AUC показывает качество по всем порогам сразу.


In [ ]:
fpr, tpr, thr = roc_curve(y_test, proba_test)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr)
plt.title("ROC curve — LogisticRegression")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.grid(True)
plt.show()

# 8) CV сравнение моделей (Tree / RF / Boosting)

Оценка: **F1** (часто лучше, чем accuracy при дисбалансе).


In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = {
    "LogReg": Pipeline([("preprocess", preprocess), ("model", LogisticRegression(max_iter=3000))]),
    "Tree": Pipeline([("preprocess", preprocess), ("model", DecisionTreeClassifier(random_state=42))]),
    "RF": Pipeline([("preprocess", preprocess), ("model", RandomForestClassifier(
        n_estimators=400, min_samples_leaf=2, random_state=42, n_jobs=-1
    ))]),
    "GB": Pipeline([("preprocess", preprocess), ("model", GradientBoostingClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
    ))]),
}

# Опционально: XGBoost, если установлен
xgb_ok = True
try:
    from xgboost import XGBClassifier
except Exception:
    xgb_ok = False

if xgb_ok:
    models["XGB"] = Pipeline([("preprocess", preprocess), ("model", XGBClassifier(
        n_estimators=500, learning_rate=0.05, max_depth=4,
        subsample=0.9, colsample_bytree=0.9,
        eval_metric="logloss",
        random_state=42, n_jobs=-1
    ))])

rows = []
for name, m in models.items():
    scores = cross_val_score(m, X_train, y_train, cv=cv, scoring="f1")
    rows.append({"model": name, "cv_f1_mean": scores.mean(), "cv_f1_std": scores.std()})
    print(f"{name:6s} | mean={scores.mean():.4f} std={scores.std():.4f} | scores={np.round(scores,4)}")

cv_table = pd.DataFrame(rows).sort_values("cv_f1_mean", ascending=False)
cv_table

# 9) GridSearchCV для кандидата (минимально)

Выбираем модель-кандидат и тюним 2–3 параметра маленькой сеткой.


In [ ]:
best_name = cv_table.iloc[0]["model"]
print("Best by CV (mean F1):", best_name)

# Часто удобно тюнить RF как старт
CANDIDATE = "RF" if "RF" in models else best_name
print("Candidate for GridSearch:", CANDIDATE)

candidate_pipe = models[CANDIDATE]

if CANDIDATE == "RF":
    param_grid = {
        "model__n_estimators": [200, 500],
        "model__max_depth": [None, 4, 6],
        "model__min_samples_leaf": [1, 2, 5],
    }
elif CANDIDATE == "GB":
    param_grid = {
        "model__n_estimators": [100, 300, 600],
        "model__learning_rate": [0.03, 0.05, 0.1],
        "model__max_depth": [2, 3, 4],
    }
elif CANDIDATE == "Tree":
    param_grid = {
        "model__max_depth": [2, 3, 4, 5, None],
        "model__min_samples_leaf": [1, 2, 5, 10],
    }
elif CANDIDATE == "LogReg":
    param_grid = {"model__C": [0.1, 1.0, 3.0, 10.0]}
elif CANDIDATE == "XGB":
    param_grid = {
        "model__max_depth": [3, 4, 5],
        "model__n_estimators": [300, 600],
        "model__learning_rate": [0.03, 0.05, 0.1],
    }
else:
    param_grid = {}

gs = GridSearchCV(candidate_pipe, param_grid=param_grid, cv=cv, scoring="f1", n_jobs=-1)
gs.fit(X_train, y_train)

print("Best params:", gs.best_params_)
print("Best CV F1 :", gs.best_score_)

# 10) Финальная модель → test → порог → save/load

Правильная логика:
- подбираем параметры на **train** через CV
- test используем **1 раз** для честной оценки


In [ ]:
best_model = gs.best_estimator_
print("Final model:", best_model)

best_model.fit(X_train, y_train)

proba_test = best_model.predict_proba(X_test)[:, 1]
m05, y_pred_05 = metrics_at_threshold(y_test.values, proba_test, thr=0.5)

print("Metrics @0.5:", m05)
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred_05))
print("\nReport:\n", classification_report(y_test, y_pred_05, digits=4))

# ROC curve
fpr, tpr, thr = roc_curve(y_test, proba_test)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr)
plt.title("ROC curve — Final model")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.grid(True)
plt.show()

# Пороговая таблица
thresholds = [0.2, 0.35, 0.5, 0.65, 0.8]
rows = []
for t in thresholds:
    m, _ = metrics_at_threshold(y_test.values, proba_test, thr=t)
    rows.append(m)
display(pd.DataFrame(rows))

# Подбор порога по F1 (демо: на test — для понимания, в проекте делайте на val)
grid = np.linspace(0.05, 0.95, 91)
best_thr, best_f1 = None, -1
for t in grid:
    m, _ = metrics_at_threshold(y_test.values, proba_test, thr=float(t))
    if m["f1"] > best_f1:
        best_f1 = m["f1"]
        best_thr = t
print("Best threshold by F1 (demo):", best_thr, "F1:", best_f1)

## 10.1) Просмотр параметров (get_params)

Полезно, чтобы увидеть, что реально внутри Pipeline, и какие гиперпараметры применились.


In [ ]:
params = best_model.get_params()
model_keys = sorted([k for k in params.keys() if k.startswith("model__")])

print("Some model__ params:")
for k in model_keys[:40]:
    print(k, "=", params[k])

## 10.2) Сохранение и загрузка модели

Сохраняем **весь Pipeline** (preprocess + модель) — это удобно и безопасно.


In [ ]:
MODEL_PATH = "classification_model.joblib"
joblib.dump(best_model, MODEL_PATH)
print("Saved model to:", MODEL_PATH)

loaded_model = joblib.load(MODEL_PATH)
print("Loaded model type:", type(loaded_model))

# Проверим, что предсказания совпадают на первых 5 строках test
p1 = best_model.predict_proba(X_test.head(5))[:, 1]
p2 = loaded_model.predict_proba(X_test.head(5))[:, 1]
print("proba original:", np.round(p1, 4))
print("proba loaded  :", np.round(p2, 4))

pred_loaded = (p2 >= 0.5).astype(int)
print("pred_loaded:", pred_loaded)

# 11) Как адаптировать под Kaggle (чек-лист)

1) Укажите `DATA_PATH`, `TARGET_COL` (и при необходимости `ID_COL`)  
2) Если target не 0/1 → сделайте `map` в 0/1  
3) Обзор: `df.head()`, `df.dtypes`, `df.isna().mean()`  
4) Мини-EDA: баланс классов, пропуски, 1–2 графика  
5) `train_test_split(..., stratify=y)`  
6) `preprocess` (num/cat) через ColumnTransformer  
7) Baseline LogisticRegression (с вероятностями)  
8) CV сравнение моделей (F1)  
9) GridSearch для кандидата  
10) Финальная оценка на test + ROC + пороги  
11) `joblib.dump` и проверка `joblib.load`  